In [29]:
# Use Braket SDK Cost Tracking to estimate the cost to run this example
from braket.tracking import Tracker
t = Tracker().start()


# general imports
import matplotlib.pyplot as plt
%matplotlib inline

# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, Gate, Instruction, circuit, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice ,AwsQuantumTask
import numpy as np

import json     

from qiskit import QuantumCircuit
from braket.ir.openqasm import Program

In [131]:

#basic non computational datas about the task
def list_task_metadata( qtask):
    meta = qtask.metadata()

    params = meta["deviceParameters"]

    print("############################")
    print("Quantum Task Metadata")
    print("Task arn:", meta["quantumTaskArn"])
    print("Task shots#:", meta["shots"])
    print("Task status:", meta["status"])
    print("Task tags:", meta["tags"])
    print("Task on device:", meta["deviceArn"])
    print("Creation date:", meta["createdAt"])
    print("End date:", meta["endedAt"])
    print("Output buket data:", meta["outputS3Directory"], " / ", meta["outputS3Bucket"])
    
    for key, value in meta.items() :
        print( key, value)

# HTTP related data for the exchange protocol(?)
def list_task_response_metadata(qtask):
    meta = qtask.metadata()

    response = meta["ResponseMetadata"]

    for key, value in response.items() :
        print( key, value)

#device terribly basic info: device arn, qbit count and qbit rewriting flag
def list_task_device_metadata(qtask):
    meta = qtask.metadata()

    dev = meta["deviceParameters"]
    devDict = json.loads(dev)

    schema = devDict["braketSchemaHeader"]
    para = devDict["paradigmParameters"]

    print("Header name:", schema["name"])
    print("Header version:", schema["version"])
    print("Simulation QBit count:", para["qubitCount"])
    print("Qubit rewriting disabled:", para["disableQubitRewiring"])

    # for key,value in para["braketSchemaHeader"].items() :
    #     print( key, value)

#essentially circuit/compilation information and measurements data
def list_task_result_metadata(qtask, task_type = None):

    result = qtask.result()

    #boring
    task_meta = result.task_metadata

    #circuit information inside this
    meta = result.additional_metadata
    circ = meta.action.source
    inputs = meta.action.inputs
    
    print("#####################")
    print("OpenQASM input program")
    print(circ)
    print("Circuit inputs:", inputs)

    if task_type == "SIMULATOR":
        print("Simulator execution duration:", meta.simulatorMetadata.executionDuration)
    elif task_type == "OQC":
        print("#####################")
        print("OpenQASM compiled program")
        print(meta.oqcMetadata.compiledProgram)
    elif task_type == "RIGETTI":
        print("#####################")
        print("OpenQASM compiled program")
        print(meta.rigettiMetadata.compiledProgram)

    # for key, value in meta.__dict__.items():
    #     print(key, value)

In [132]:
#retrieve example Simulator Task

sim_task_id = "arn:aws:braket:eu-west-2:058017682654:quantum-task/40d46f75-6557-451b-b223-1c5abd8b1a51" 
task = AwsQuantumTask(arn = sim_task_id)

list_task_result_metadata(task, task_type = "SIMULATOR")


#####################
OpenQASM input program
OPENQASM 3.0;
bit[2] b;
qubit[2] q;
x q[1];
h q[0];
h q[1];
#pragma braket unitary([[0, 1.0, 0, 0], [1.0, 0, 0, 0], [0, 0, 0, 1.0], [0, 0, 1.0, 0]]) q[0], q[1]
h q[0];
h q[1];
b[0] = measure q[1];
b[1] = measure q[0];
Circuit inputs: None
Simulator execution duration: 9


In [133]:

#lucy example Deutsch task

lucy_task_id = "arn:aws:braket:eu-west-2:058017682654:quantum-task/a2b43a8e-3e5e-4776-bfc4-39ae0cb9a986" 
task = AwsQuantumTask(arn = lucy_task_id)

list_task_result_metadata(task, task_type = "OQC")

#####################
OpenQASM input program
OPENQASM 3.0;
bit[2] b;
qubit[2] q;
x q[1];
h q[0];
h q[1];
cnot q[0], q[1];
h q[0];
h q[1];
b[0] = measure q[1];
b[1] = measure q[0];
Circuit inputs: None
#####################
OpenQASM compiled program
OPENQASM 2.0;
include "qelib1.inc";

qreg node[6];
creg b[2];
u3(1.5*pi,-0.5*pi,4.0*pi) node[4];
u3(1.5*pi,0.0*pi,4.0*pi) node[5];
cx node[4],node[5];
u3(0.5*pi,0.0*pi,0.5*pi) node[4];
u3(0.5*pi,0.0*pi,1.0*pi) node[5];
measure node[5] -> b[0];
measure node[4] -> b[1];



In [28]:
circ = QuantumCircuit().from_qasm_str("""OPENQASM 2.0;
include "qelib1.inc";

qreg node[6];
creg b[2];
u3(1.5*pi,-0.5*pi,4.0*pi) node[4];
u3(1.5*pi,0.0*pi,4.0*pi) node[5];
cx node[4],node[5];
u3(0.5*pi,0.0*pi,0.5*pi) node[4];
u3(0.5*pi,0.0*pi,1.0*pi) node[5];
measure node[5] -> b[0];
measure node[4] -> b[1];""")

circ.draw()

node_0: ────────────────────────────────────────────────
                                                        
node_1: ────────────────────────────────────────────────
                                                        
node_2: ────────────────────────────────────────────────
                                                        
node_3: ────────────────────────────────────────────────
        ┌──────────────────┐     ┌───────────────┐   ┌─┐
node_4: ┤ U3(3π/2,-π/2,4π) ├──■──┤ U3(π/2,0,π/2) ├───┤M├
        └┬───────────────┬─┘┌─┴─┐└┬─────────────┬┘┌─┐└╥┘
node_5: ─┤ U3(3π/2,0,4π) ├──┤ X ├─┤ U3(π/2,0,π) ├─┤M├─╫─
         └───────────────┘  └───┘ └─────────────┘ └╥┘ ║ 
   b_0: ═══════════════════════════════════════════╩══╬═
                                                      ║ 
   b_1: ══════════════════════════════════════════════╩═

In [32]:
program = Program(source="""OPENQASM 3.0;
bit[2] b;
qubit[2] q;
x q[1];
h q[0];
h q[1];
cnot q[0], q[1];
h q[0];
h q[1];
b[0] = measure q[1];
b[1] = measure q[0];
Circuit inputs: None)""")
print(program)


braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program', version='1') source='OPENQASM 3.0;\nbit[2] b;\nqubit[2] q;\nx q[1];\nh q[0];\nh q[1];\ncnot q[0], q[1];\nh q[0];\nh q[1];\nb[0] = measure q[1];\nb[1] = measure q[0];\nCircuit inputs: None)' inputs=None
